<a href="https://colab.research.google.com/github/mayank22415/chatbot/blob/main/neuralTrainingMinu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import json
import numpy as np
import pickle
import string
import nltk


In [ ]:

!pip install tensorflow


In [ ]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
#f=open('intents.json','r',errors = 'ignore')
#raw=f.read()
#raw=raw.lower()
nltk.download('punkt_tab')
nltk.download('wordnet')
#sent_tokens = nltk.sent_tokenize(raw)
#word_tokens = nltk.word_tokenize(raw)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
intents = json.loads(open('intents.json').read())

In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

In [ ]:
for intent in intents['intents']:
   for pattern in intent['patterns']:
       word_list = nltk.word_tokenize(pattern)
       words.extend(word_list)
       documents.append((word_list,intent['tag']))
       if intent['tag'] not in classes:
           classes.append(intent['tag'])


In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word[0] not in ignore_letters]
words = sorted(set(words))

In [ ]:
#print(words)

In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
training = []
output_empty = [0]*len(classes)

now i vectorizing the document if word is present it will else it will be zero eg. [how,are,you,doing]
document1 - how are you
vectorization [1,1,1,0]

In [ ]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [ lemmatizer.lemmatize(word.lower())  for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])


In [ ]:
random.shuffle(training)
training = np.array(training, dtype=object)

training_x = list(training[:, 0])
training_y = list(training[:, 1])

training_x = np.array(training_x)

In [ ]:
model = Sequential()
model.add(Dense(128,input_shape=(len(training_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(training_y[0]),activation='softmax'))

In [ ]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [ ]:
hist = model.fit(np.array(training_x),np.array(training_y),epochs=300,batch_size=5,verbose=1)
model.save('my_model.keras')
#model.save('chatbot.h5',hist)
print('done')

In [ ]:
import json
import random
import numpy as np
import nltk
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer

# Load the model
model = load_model('my_model.keras')

# Load intents and other data
with open('intents.json') as json_data:
    intents = json.load(json_data)

# Load saved words and classes (your vocabulary and labels)
import pickle
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

lemmatizer = WordNetLemmatizer()

# Preprocessing function
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Create a bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

# Predict intent
def predict_class(sentence):
    bow_vector = bow(sentence, words, show_details=False)
    res = model.predict(np.array([bow_vector]))[0]
    threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > threshold]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Get response from intents
def get_response(ints, intents_json):
    if len(ints) == 0:
        return "I didn't understand that. Can you rephrase?"
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])

# Chat function
def chatbot_response(msg):
    ints = predict_class(msg)
    res = get_response(ints, intents)
    return res

# Run chatbot
print("hii i am minu! Type 'quit' to stop.")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print("Bot:", response)
